# TRAIN資料前處理

In [ ]:
!pip install -q optuna optuna-integration[tfkeras]
import os, random, numpy as np, pandas as pd, joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import shap
from tqdm import tqdm

GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
MODEL_DIR = '/kaggle/working'
SEQ_LEN = 32
max_rul = 130
window_size = 12
threshold = 0.2

columns = ['unit', 'time', 'op1', 'op2', 'op3'] + [f's{i}' for i in range(1, 22)]
train = pd.read_csv('/kaggle/input/cmapssdata/train_FD003.txt', sep='\s+', header=None)
train.columns = columns

def generate_piecewise_rul(df, feature_cols, window_size=12, threshold=0.2, max_rul=130):
    result_dfs = []
    for unit_id, group in df.groupby('unit'):
        group = group.sort_values('time').reset_index(drop=True)
        sensor_data = group[feature_cols].values
        num_cycles = len(group)
        num_windows = num_cycles // window_size
        if num_windows < 3:
            irul = num_cycles
            group['iRUL'] = irul
            group['RUL_piecewise'] = np.clip(irul - group['time'], 0, max_rul)
            result_dfs.append(group)
            continue
        centroids = [np.mean(sensor_data[i*window_size:(i+1)*window_size], axis=0) for i in range(num_windows)]
        base = centroids[0]
        degradation_found = False
        for i in range(2, num_windows):
            dist_sq = np.sum((centroids[i] - base) ** 2)
            if dist_sq >= threshold:
                degradation_start = i * window_size
                irul = num_cycles - degradation_start
                degradation_found = True
                break
        if not degradation_found:
            irul = num_cycles
        rul_piecewise = [irul if t <= (num_cycles - irul) else irul - (t - (num_cycles - irul)) for t in range(num_cycles)]
        group['iRUL'] = irul
        group['RUL_piecewise'] = np.clip(rul_piecewise, 0, max_rul)
        result_dfs.append(group)
    return pd.concat(result_dfs, ignore_index=True)

sensor_cols = [f's{i}' for i in range(1, 22)]
sensor_cols_keep = [col for col in sensor_cols if train[col].std() >= 1e-3]

def z_score_filter(df, cols, threshold=4.0):
    for col in cols:
        mean = df[col].mean()
        std = df[col].std()
        z = (df[col] - mean) / std
        df.loc[z.abs() > threshold, col] = mean
    return df

train = z_score_filter(train, sensor_cols_keep)
train = generate_piecewise_rul(train, sensor_cols_keep, window_size, threshold, max_rul)

sample_n = min(2000, len(train))
train_sample = train.sample(n=sample_n, random_state=GLOBAL_SEED)
rf = RandomForestRegressor(n_estimators=100, random_state=GLOBAL_SEED, n_jobs=-1)
rf.fit(train_sample[sensor_cols_keep], train_sample['RUL_piecewise'])

shap_sample_n = min(200, len(train_sample))
shap_sample_idx = np.random.choice(len(train_sample), shap_sample_n, replace=False)
shap_X = train_sample[sensor_cols_keep].iloc[shap_sample_idx]
explainer = shap.Explainer(rf, shap_X)
shap_values = explainer(shap_X, check_additivity=False)
mean_shap = np.abs(shap_values.values).mean(axis=0)
shap_scores = pd.Series(mean_shap, index=sensor_cols_keep).sort_values(ascending=False)
top8_shap = shap_scores.head(8).index.tolist()
corrs = train[sensor_cols_keep + ['RUL_piecewise']].corr()['RUL_piecewise'].abs().sort_values(ascending=False)
top8_pearson = corrs.drop('RUL_piecewise').head(8).index.tolist()
selected_sensors = [x for x in top8_shap if x in top8_pearson]
for x in top8_pearson:
    if x not in selected_sensors:
        selected_sensors.append(x)
    if len(selected_sensors) == 8:
        break

features = selected_sensors + ['op1', 'op2', 'op3']

def multi_exponential_smoothing(series, alphas=[0.1, 0.3]):
    results = []
    for alpha in alphas:
        smoothed = [series.iloc[0]]
        for n in range(1, len(series)):
            smoothed.append(alpha * series.iloc[n] + (1 - alpha) * smoothed[-1])
        results.append(pd.Series(smoothed, index=series.index))
    return sum(results) / len(results)

for col in features:
    train[col] = train.groupby('unit')[col].transform(lambda x: multi_exponential_smoothing(x)).astype('float64')
for col in selected_sensors:
    train[f'{col}_diff'] = train.groupby('unit')[col].diff().fillna(0)
final_features = features + [f'{col}_diff' for col in selected_sensors]

scaler = StandardScaler()
train[final_features] = scaler.fit_transform(train[final_features])
joblib.dump(scaler, f'{MODEL_DIR}/fd003_scaler_preprocessed_Diff.joblib')
joblib.dump(final_features, f'{MODEL_DIR}/fd003_feature_names_Diff.pkl')
train.to_csv(f'{MODEL_DIR}/fd003_train_with_piecewise_rul_Diff.csv', index=False)

X_list, y_list, unit_list = [], [], []
for unit in tqdm(train['unit'].unique(), desc='產生滑動視窗'):
    df_unit = train[train['unit'] == unit]
    arr = df_unit[final_features].values
    rul_piecewise = df_unit['RUL_piecewise'].values
    if len(arr) < SEQ_LEN:
        continue
    for i in range(len(arr) - SEQ_LEN + 1):
        X_list.append(arr[i:i + SEQ_LEN])
        y_list.append(rul_piecewise[i + SEQ_LEN - 1])
        unit_list.append(unit)
X_all = np.stack(X_list).astype(np.float32)
y_all = np.array(y_list).astype(np.float32)
unit_all = np.array(unit_list).astype(int)
np.save(f'{MODEL_DIR}/fd003_X_all_Diff.npy', X_all)
np.save(f'{MODEL_DIR}/fd003_y_all_Diff.npy', y_all)
np.save(f'{MODEL_DIR}/fd003_unit_all_Diff.npy', unit_all)
print(f"滑動視窗完成: X_all={X_all.shape}, y_all={y_all.shape}")


# Stage1 Encoder 預訓練

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import numpy as np
import os
import matplotlib.pyplot as plt

SEQ_LEN = 32
D = 16
T = 1000
MODEL_DIR = '/kaggle/working'

X_all = np.load(f'{MODEL_DIR}/fd003_X_all_Diff.npy')

def cbam_block(inputs, reduction_ratio=8):
    channel = int(inputs.shape[-1])
    avg_pool = layers.GlobalAveragePooling1D(keepdims=True)(inputs)
    max_pool = layers.GlobalMaxPooling1D(keepdims=True)(inputs)
    dense = layers.Dense(channel // reduction_ratio, activation='relu')
    dense_out = layers.Dense(channel)
    avg_out = dense_out(dense(avg_pool))
    max_out = dense_out(dense(max_pool))
    channel_attention = layers.Activation('sigmoid')(layers.Add()([avg_out, max_out]))
    channel_refined = layers.Multiply()([inputs, channel_attention])
    avg_pool_spatial = layers.GlobalAveragePooling1D(keepdims=True)(channel_refined)
    max_pool_spatial = layers.GlobalMaxPooling1D(keepdims=True)(channel_refined)
    concat = layers.Concatenate(axis=-1)([avg_pool_spatial, max_pool_spatial])
    spatial_attention = layers.Conv1D(1, 7, padding='same', activation='sigmoid')(concat)
    refined = layers.Multiply()([channel_refined, spatial_attention])
    return refined

def resblock(x, filters, kernel_size=3, stride=1, add_noise=0.01, dropout=0.15):
    shortcut = x
    x = layers.Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GaussianNoise(add_noise)(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = cbam_block(x)
    if int(shortcut.shape[-1]) != filters:
        shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

def align_and_concat(x1, x2):
    t1 = x1.shape[1]
    t2 = x2.shape[1]
    minlen = min(t1, t2)
    if t1 > minlen:
        x1 = layers.Cropping1D((0, t1 - minlen))(x1)
    if t2 > minlen:
        x2 = layers.Cropping1D((0, t2 - minlen))(x2)
    return layers.Concatenate()([x1, x2])

def build_unet_encoder(input_dim, embed_dim, seq_len, base_filters=64, depth=4, drop_enc=0.18, noise_enc=0.06):
    input_x = Input(shape=(seq_len, input_dim), name='input_1')
    input_tau = Input(shape=(embed_dim,), name='input_2')
    x = layers.GaussianNoise(noise_enc)(input_x)
    t_proj = layers.Dense(input_dim)(input_tau)
    t_proj_exp = layers.Reshape((1, input_dim))(t_proj)
    x = layers.Add()([x, t_proj_exp])
    skips = []
    for d in range(depth):
        filters = base_filters * (2 ** d)
        x = resblock(x, filters, add_noise=noise_enc, dropout=drop_enc)
        skips.append(x)
        if d != depth - 1:
            x = layers.MaxPooling1D(2)(x)
    filters = base_filters * (2 ** depth)
    x = resblock(x, filters, add_noise=noise_enc, dropout=drop_enc)
    for d in reversed(range(depth)):
        x = layers.UpSampling1D(2)(x)
        x = align_and_concat(x, skips[d])
        x = resblock(x, base_filters * (2 ** d), add_noise=noise_enc, dropout=drop_enc)
    output = layers.Conv1D(input_dim, 1, padding='same')(x)
    model = Model([input_x, input_tau], output)
    return model

def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = np.linspace(0, timesteps, steps)
    f = np.cos(((x / timesteps) + s) / (1 + s) * np.pi / 2) ** 2
    alphas_cumprod = f / f[0]
    betas = np.clip(1 - (alphas_cumprod[1:] / alphas_cumprod[:-1]), 0, 0.999)
    return betas

betas = cosine_beta_schedule(T, s=0.008)
alphas = 1 - betas
alphas_cumprod = np.cumprod(alphas)

def get_timestep_embedding(timesteps, dim=D):
    timesteps = tf.convert_to_tensor(timesteps, dtype=tf.float32)
    timesteps = tf.reshape(timesteps, [-1, 1])
    half_dim = dim // 2
    emb = tf.math.log(10000.0) / (half_dim - 1)
    emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
    emb = timesteps * emb
    emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
    return emb

def diffusion_dataset(X, batch_size, T, alphas_cumprod, D):
    def generator():
        dataset_size = len(X)
        while True:
            idxs = np.random.permutation(dataset_size)
            for i in range(0, dataset_size, batch_size):
                batch_idx = idxs[i:i+batch_size]
                x_start = X[batch_idx]
                b = len(x_start)
                t = np.random.randint(1, T + 1, size=b)
                tau = get_timestep_embedding(t, D).numpy()
                noise = np.random.randn(*x_start.shape).astype(np.float32)
                sqrt_alpha = np.sqrt(alphas_cumprod[t - 1])[:, None, None]
                sqrt_one_minus_alpha = np.sqrt(1 - alphas_cumprod[t - 1])[:, None, None]
                x_t = sqrt_alpha * x_start + sqrt_one_minus_alpha * noise
                yield {"input_1": x_t, "input_2": tau}, noise
    output_signature = (
        {"input_1": tf.TensorSpec(shape=(None, SEQ_LEN, X.shape[-1]), dtype=tf.float32),
         "input_2": tf.TensorSpec(shape=(None, D), dtype=tf.float32)},
        tf.TensorSpec(shape=(None, SEQ_LEN, X.shape[-1]), dtype=tf.float32)
    )
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

BATCH_SIZE = 128
EPOCHS_STAGE1 = 300

train_ds = diffusion_dataset(X_all, BATCH_SIZE, T, alphas_cumprod, D)
steps_per_epoch = len(X_all) // BATCH_SIZE

unet_encoder = build_unet_encoder(
    input_dim=X_all.shape[-1], embed_dim=D, seq_len=SEQ_LEN,
    base_filters=64, depth=4, drop_enc=0.18, noise_enc=0.06
)
unet_encoder.compile(optimizer=tf.keras.optimizers.Adam(5e-4), loss='mse')

history = unet_encoder.fit(
    train_ds,
    epochs=EPOCHS_STAGE1,
    steps_per_epoch=steps_per_epoch,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)]
)

print("Stage1 Loss History:")
print(history.history['loss'])

plt.figure(figsize=(7,4))
plt.plot(history.history['loss'], marker='o')
plt.title("Stage1 Diffusion Encoder Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/fd003_stage1_encoder_loss_curve_Diff.png')
plt.show()

print(f"最後Loss值: {history.history['loss'][-1]:.4f}")

weights = unet_encoder.get_weights()
for i, w in enumerate(weights):
    print(f"Layer {i} weight mean: {np.mean(w):.4f}, std: {np.std(w):.4f}")

gap = layers.GlobalAveragePooling1D()(unet_encoder.layers[-2].output)
encoder_embedding_model = Model(unet_encoder.inputs, gap)
unet_encoder.save(f'{MODEL_DIR}/fd003_stage1_encoder_full_Diff.keras')
encoder_embedding_model.save(f'{MODEL_DIR}/fd003_stage1_encoder_embedding_Diff.keras')
print("✅ FD003 Stage1 Encoder與Embedding模型已儲存")
print(f"✅ Loss曲線儲存於 {MODEL_DIR}/fd003_stage1_encoder_loss_curve_Diff.png")


# Stage2 Optuna 搜尋 + RUL MLP

In [ ]:
import os
import shutil
import optuna
from optuna.integration import TFKerasPruningCallback
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.optimizers import AdamW
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import numpy as np
import gc

SEQ_LEN, D, T = 32, 16, 1000
GLOBAL_SEED = 42

MODEL_DIR = '/kaggle/working'
model_path = '/kaggle/working/fd003_stage2_joint_model_best_Diff.keras'
optuna_csv_path = '/kaggle/working/fd003_optuna_search_results_Diff.csv'

X_all = np.load(f'{MODEL_DIR}/fd003_X_all_Diff.npy')
y_all = np.load(f'{MODEL_DIR}/fd003_y_all_Diff.npy')
unit_all = np.load(f'{MODEL_DIR}/fd003_unit_all_Diff.npy')

unit_ids = np.unique(unit_all)
n_unit = len(unit_ids)
split = int(n_unit * 0.8)
train_units = set(unit_ids[:split])
val_units = set(unit_ids[split:])

X_train, y_train, X_val, y_val = [], [], [], []
for x, y, u in zip(X_all, y_all, unit_all):
    if u in train_units:
        X_train.append(x)
        y_train.append(y)
    elif u in val_units:
        X_val.append(x)
        y_val.append(y)
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_val = np.array(X_val, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)

def get_timestep_embedding_np(timesteps, dim):
    timesteps = np.array(timesteps).reshape(-1, 1)
    half_dim = dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = np.exp(np.arange(half_dim) * -emb)
    emb = timesteps * emb
    emb = np.concatenate([np.sin(emb), np.cos(emb)], axis=1)
    return emb.astype(np.float32)

t_train = np.random.randint(1, T + 1, size=len(X_train))
tau_train = get_timestep_embedding_np(t_train, D)
t_val = np.full(len(X_val), SEQ_LEN)
tau_val = get_timestep_embedding_np(t_val, D)

def mixup(x, t, y, alpha=0.2):
    idx = np.random.permutation(len(x))
    lam = np.random.beta(alpha, alpha)
    x_mix = lam * x + (1 - lam) * x[idx]
    t_mix = lam * t + (1 - lam) * t[idx]
    y_mix = lam * y + (1 - lam) * y[idx]
    return x_mix, t_mix, y_mix

def build_rul_predictor(input_dim, n_layers, hidden_units, dropout=0.18):
    x_in = Input(shape=(input_dim,))
    x = layers.BatchNormalization()(x_in)
    for i in range(n_layers):
        x = layers.Dense(hidden_units[i], activation='relu')(x)
        x = layers.BatchNormalization()(x)
        if i > 0:
            x = layers.Dropout(dropout)(x)
    y_out = layers.Dense(1, activation='linear')(x)
    return Model(x_in, y_out)

def objective(trial):
    tf.keras.backend.clear_session()
    gc.collect()
    encoder_embedding_model = tf.keras.models.load_model(
        f'{MODEL_DIR}/fd003_stage1_encoder_embedding_Diff.keras', compile=False)
    encoder_embedding_model.trainable = True

    n_layers = trial.suggest_int("n_layers", 2, 3)           # 建議2~3層
    hidden_units = [
        trial.suggest_int("h1", 192, 320, step=32),          # 更寬範圍
        trial.suggest_int("h2", 96, 192, step=16),
        trial.suggest_int("h3", 48, 96, step=8),
        trial.suggest_int("h4", 16, 48, step=8),
    ]
    lr = trial.suggest_float("learning_rate", 7e-5, 2e-4, log=True)
    dropout = trial.suggest_float("dropout", 0.15, 0.21)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    patience_es = trial.suggest_int("patience_es", 12, 18)
    patience_rlr = trial.suggest_int("patience_rlr", 5, 9)
    lambda_high = trial.suggest_float("lambda_high", 0.06, 0.10)
    mixup_alpha = trial.suggest_float("mixup_alpha", 0.13, 0.20)

    def custom_rul_loss(lambda_high=lambda_high):
        def loss(y_true, y_pred):
            base = tf.reduce_mean(tf.square(tf.clip_by_value(y_pred, 0, 130) - y_true))
            high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
            return base + lambda_high * high_bias
        return loss

    input_x = Input(shape=(SEQ_LEN, X_train.shape[-1]), name="stage2_input_x")
    input_t = Input(shape=(D,), name="stage2_input_t")
    encoded = encoder_embedding_model([input_x, input_t])
    mlp = build_rul_predictor(encoded.shape[-1], n_layers, hidden_units, dropout)
    output = mlp(encoded)
    joint_model = Model([input_x, input_t], output)
    joint_model.compile(optimizer=AdamW(learning_rate=lr), loss=custom_rul_loss())

    x_tr, t_tr, y_tr = X_train, tau_train, y_train
    if mixup_alpha > 0:
        x_tr, t_tr, y_tr = mixup(X_train, tau_train, y_train, alpha=mixup_alpha)

    try:
        history = joint_model.fit(
            [x_tr, t_tr], y_tr,
            validation_data=([X_val, tau_val], y_val),
            epochs=50,
            batch_size=batch_size,
            verbose=1,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=patience_es, restore_best_weights=True),
                tf.keras.callbacks.ReduceLROnPlateau(patience=patience_rlr, factor=0.5, min_lr=1e-7),
                TFKerasPruningCallback(trial, "val_loss")
            ]
        )
        y_pred = joint_model.predict([X_val, tau_val], batch_size=batch_size).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    except Exception as e:
        print(f"[OOM or ERROR] {e}")
        rmse = 1e9

    del joint_model, mlp, encoded, input_x, input_t, encoder_embedding_model
    tf.keras.backend.clear_session()
    gc.collect()
    return rmse

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=GLOBAL_SEED))
study.optimize(objective, n_trials=80, timeout=7200)

best = study.best_params
print(f"✅ Optuna最佳參數: {best}")

tf.keras.backend.clear_session()
gc.collect()
encoder_embedding_model = tf.keras.models.load_model(
    f'{MODEL_DIR}/fd003_stage1_encoder_embedding_Diff.keras', compile=False)
encoder_embedding_model.trainable = True

best_hidden_units = [
    best["h1"],
    best["h2"],
    best["h3"],
    best["h4"],
]
input_x = Input(shape=(SEQ_LEN, X_train.shape[-1]), name="stage2_input_x_final")
input_t = Input(shape=(D,), name="stage2_input_t_final")
encoded = encoder_embedding_model([input_x, input_t])
mlp = build_rul_predictor(
    encoded.shape[-1],
    n_layers=best["n_layers"],
    hidden_units=best_hidden_units,
    dropout=best["dropout"],
)
output = mlp(encoded)

def custom_rul_loss(lambda_high=best["lambda_high"]):
    def loss(y_true, y_pred):
        base = tf.reduce_mean(tf.square(tf.clip_by_value(y_pred, 0, 130) - y_true))
        high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
        return base + lambda_high * high_bias
    return loss

final_model = Model([input_x, input_t], output)
final_model.compile(optimizer=AdamW(learning_rate=best['learning_rate']), loss=custom_rul_loss())

if best.get('mixup_alpha', 0) > 0:
    x_tr, t_tr, y_tr = mixup(X_train, tau_train, y_train, alpha=best['mixup_alpha'])
else:
    x_tr, t_tr, y_tr = X_train, tau_train, y_train

final_model.fit(
    [x_tr, t_tr], y_tr,
    validation_data=([X_val, tau_val], y_val),
    epochs=100,
    batch_size=best['batch_size'],
    verbose=2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=best['patience_es'], restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=best['patience_rlr'], factor=0.5, min_lr=1e-7)
    ]
)

if os.path.exists(model_path):
    if os.path.isfile(model_path):
        os.remove(model_path)
    else:
        shutil.rmtree(model_path)
final_model.save(model_path)

study.trials_dataframe().to_csv(optuna_csv_path, index=False)


# Test 資料前處理

In [ ]:
import numpy as np
import pandas as pd
import joblib

MODEL_DIR = '/kaggle/working'
SEQ_LEN = 32

scaler = joblib.load(f'{MODEL_DIR}/fd003_scaler_preprocessed_Diff.joblib')
feature_names = joblib.load(f'{MODEL_DIR}/fd003_feature_names_Diff.pkl')

columns = ['unit', 'time', 'op1', 'op2', 'op3'] + [f's{i}' for i in range(1, 22)]
test = pd.read_csv('/kaggle/input/cmapssdata/test_FD003.txt', sep='\s+', header=None)
test.columns = columns
rul_truth = pd.read_csv('/kaggle/input/cmapssdata/RUL_FD003.txt', header=None, names=['RUL'])

def exponential_smoothing(series, alpha=0.2):
    result = [series.iloc[0]]
    for n in range(1, len(series)):
        result.append(alpha * series.iloc[n] + (1 - alpha) * result[-1])
    return pd.Series(result, index=series.index)

for col in feature_names:
    if col.endswith('_diff'):
        continue
    if col in test.columns:
        test[col] = test.groupby('unit')[col].transform(lambda x: exponential_smoothing(x, alpha=0.2))
for col in feature_names:
    if col.endswith('_diff'):
        base_col = col.replace('_diff', '')
        if base_col in test.columns:
            test[col] = test.groupby('unit')[base_col].diff().fillna(0)
        else:
            test[col] = 0.0
for col in feature_names:
    if col not in test.columns:
        test[col] = 0.0

test[feature_names] = scaler.transform(test[feature_names])

X_test_list, timestep_list, units_list = [], [], []
for unit in sorted(test['unit'].unique()):
    df_unit = test[test['unit'] == unit]
    arr = df_unit[feature_names].values
    if len(arr) >= SEQ_LEN:
        X_test_list.append(arr[-SEQ_LEN:])
        timestep_list.append(df_unit['time'].values[-1])
        units_list.append(unit)
X_test = np.stack(X_test_list).astype(np.float32)

def get_timestep_embedding_np(timesteps, dim):
    timesteps = np.array(timesteps).reshape(-1, 1)
    half_dim = dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = np.exp(np.arange(half_dim) * -emb)
    emb = timesteps * emb
    emb = np.concatenate([np.sin(emb), np.cos(emb)], axis=1)
    return emb.astype(np.float32)

D = 16
tau_test = get_timestep_embedding_np(timestep_list, D)
y_test = rul_truth['RUL'].values

np.save(f'{MODEL_DIR}/fd003_X_test_Diff.npy', X_test)
np.save(f'{MODEL_DIR}/fd003_tau_test_Diff.npy', tau_test)
np.save(f'{MODEL_DIR}/fd003_units_list_Diff.npy', np.array(units_list))
np.save(f'{MODEL_DIR}/fd003_y_test_Diff.npy', y_test)

print(f"✅ Test視窗數: {len(X_test)}")
print(f"🧪 對齊unit數量: {len(np.unique(units_list))}")


# Test 評估與視覺化

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from tensorflow.keras.models import load_model
import tensorflow as tf

MODEL_DIR = '/kaggle/working'
X_test = np.load(f'{MODEL_DIR}/fd003_X_test_Diff.npy')
tau_test = np.load(f'{MODEL_DIR}/fd003_tau_test_Diff.npy')
units_list = np.load(f'{MODEL_DIR}/fd003_units_list_Diff.npy').astype(int)
rul_truth = pd.read_csv('/kaggle/input/cmapssdata/RUL_FD003.txt', header=None, names=['RUL'])
y_true = rul_truth.loc[units_list - 1, 'RUL'].values

optuna_result_csv = f'{MODEL_DIR}/fd003_optuna_search_results_Diff.csv'
optuna_df = pd.read_csv(optuna_result_csv)
if 'params_lambda_high' in optuna_df.columns:
    best_lambda_high = optuna_df.loc[optuna_df['value'].idxmin()]['params_lambda_high']
else:
    best_lambda_high = 0.1

def custom_rul_loss(lambda_high=best_lambda_high):
    def loss(y_true, y_pred):
        base = tf.reduce_mean(tf.square(tf.clip_by_value(y_pred, 0, 130) - y_true))
        high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
        return base + lambda_high * high_bias
    return loss

joint_model = load_model(f'{MODEL_DIR}/fd003_stage2_joint_model_best_Diff.keras',
                         custom_objects={'loss': custom_rul_loss()})

y_pred = joint_model.predict([X_test, tau_test], verbose=1).flatten()
y_pred = np.clip(y_pred, 0, 130)

def nasa_score(y_true, y_pred):
    score = 0
    for true, pred in zip(y_true, y_pred):
        d = pred - true
        if d < 0:
            score += np.exp(-d / 13) - 1
        else:
            score += np.exp(d / 10) - 1
    return score

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
score = nasa_score(y_true, y_pred)
print(f'✅ Test RMSE: {rmse:.4f}')
print(f'✅ Test NASA Score: {score:.4f}')

df_out = pd.DataFrame({
    'unit': units_list,
    'True_RUL': y_true,
    'Pred_RUL': y_pred,
    'Error': y_pred - y_true
})
df_out.to_csv(f'{MODEL_DIR}/fd003_test_pred_result_Diff.csv', index=False)
print(f"📁 預測結果已儲存：{MODEL_DIR}/fd003_test_pred_result_Diff.csv")

plt.figure(figsize=(8, 5))
plt.plot(y_true, label='True RUL', marker='o', alpha=0.8)
plt.plot(y_pred, label='Predicted RUL', marker='x', alpha=0.8)
plt.xlabel('Test Unit (Engine)')
plt.ylabel('RUL')
plt.legend()
plt.title(f'Predicted vs. True RUL (Test)\nRMSE={rmse:.2f} | NASA Score={score:.2f}')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/fd003_test_pred_vs_true_rul_Diff.png')
plt.close()

plt.figure(figsize=(7,5))
plt.scatter(y_true, y_pred - y_true, alpha=0.75, c='royalblue', edgecolor='k')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('True RUL')
plt.ylabel('Prediction Error (Predicted - True)')
plt.title('Residual Distribution (Error vs True RUL)')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/fd003_test_residual_scatter_Diff.png')
plt.close()

plt.figure(figsize=(7,4))
plt.hist(y_pred - y_true, bins=25, color='skyblue', edgecolor='black')
plt.title('Prediction Error Histogram')
plt.xlabel('Prediction Error (Predicted - True)')
plt.ylabel('Count')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/fd003_test_error_histogram_Diff.png')
plt.close()

try:
    encoder = load_model(f'{MODEL_DIR}/fd003_stage1_encoder_embedding_Diff.keras', compile=False)
    features = encoder.predict([X_test, tau_test])
    features_2d = TSNE(n_components=2, random_state=42).fit_transform(features)
    plt.figure(figsize=(7,6))
    sc = plt.scatter(features_2d[:,0], features_2d[:,1], c=y_true, cmap='viridis', s=20)
    plt.colorbar(sc, label='True RUL')
    plt.title("t-SNE of Encoder Embedding Features (Test set)")
    plt.xlabel("t-SNE Dim 1")
    plt.ylabel("t-SNE Dim 2")
    plt.savefig(f'{MODEL_DIR}/fd003_test_encoder_tsne_Diff.png')
    plt.close()
except Exception as e:
    print(f"[t-SNE 可視化失敗] {e}")

print(f"✅ 圖片已儲存：")
print(f" - {MODEL_DIR}/fd003_test_pred_vs_true_rul_Diff.png")
print(f" - {MODEL_DIR}/fd003_test_residual_scatter_Diff.png")
print(f" - {MODEL_DIR}/fd003_test_error_histogram_Diff.png")
print(f" - {MODEL_DIR}/fd003_test_encoder_tsne_Diff.png")
